In [2]:
### Importing ###
from numpy import exp, array, random, dot #imports linear algebra operations/types
import numpy as np #operations in numpy
from scipy import misc #misc. operations in scipy
import skimage
import math 
from __future__ import with_statement

In [3]:
### Get picture data ###
arr1 = np.divide(misc.imread('save_save_save_america02.png'),255) #read 1st image, divide by 255 for number between 0 and 1
arr1r = arr1[:,:,0] #reds
arr1g = arr1[:,:,1] #greens
arr1b = arr1[:,:,2] #blues
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [4]:
### Hyper-parameters ###
layerSize = 3 #number of filters in a layer
layerNum = 6 #number of layers
filterSize = 10 #size of filter
epochs = 10 #number of times trained
filters = [] #init filter
targets = np.ones((len(arr1r)*len(arr1r[0])*layerSize)) #target values

In [5]:
### Variables based on hyper-params ###
filterNum = layerSize**2 #number of filters per layer
totalFilterNum = filterNum*layerNum #total number of filters

In [6]:
### Set filters ###
def setFilters():
    global filters #Filters is a global variable
    filters = 0.01*random.random((layerNum, filterNum, filterSize, filterSize))-0.005 #Set filters to random 4d array
setFilters()
print(filters)

[[[[ -4.35255160e-03  -4.90301029e-03   1.83489298e-03 ...,
      3.09952027e-03  -1.37705991e-03  -4.90320259e-03]
   [ -3.51796214e-03  -3.04091345e-03  -3.73861712e-03 ...,
      1.70603187e-03  -3.63017637e-03  -4.74448311e-03]
   [ -3.61643057e-03  -1.48126615e-03  -2.37255616e-03 ...,
     -2.20924481e-03  -7.94054304e-04   1.60058010e-03]
   ..., 
   [  4.08652433e-03  -4.31025002e-03  -2.38691231e-04 ...,
     -4.40893835e-03  -4.03144567e-03   4.71336657e-03]
   [  4.60767184e-03   1.90465177e-03  -1.24639067e-04 ...,
     -1.17234087e-03  -4.25260690e-03   1.96921059e-03]
   [  3.55628360e-03  -4.14076755e-03   1.10889717e-03 ...,
     -4.09661827e-03  -7.99299098e-04  -2.83429469e-03]]

  [[  1.80503610e-03  -1.55629126e-03  -3.01370642e-03 ...,
      4.17803651e-03  -3.49709479e-03   4.60749223e-03]
   [  1.70820661e-03   3.66861658e-03  -4.72385075e-03 ...,
     -7.28051912e-04  -8.78538681e-04   4.38767616e-03]
   [ -1.78993683e-03   4.81479054e-03  -3.50977814e-03 ...,
 

In [38]:
### Helper functions ###
#Activation function (Leaky ReLU)
def activation(x, deriv=False, leak=-0.01, bias=0.01):
    new_x=np.copy(array(x)) #Create a copy of x
    if deriv: #If you want to activate derivative
        #Calculation of derivative of leaky ReLU
        new_x[x > 0] = 1 
        new_x[x < 0] = leak
        return new_x
#     new_x += bias
    #Calculation of leaky ReLU
    new_x[x < 0] *= leak
    new_x = x * (x > 0)
    return new_x
#Sum with no negatives (not used)
'''
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
'''
#Apply cross-correlation for 1 index
def apply(filterArr, img, row, col):
    value = 0 #Init value
    halfFil = int(filterSize/2) #Half of filterSize
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0)) #Pad image with 0's
    for x in range(filterSize): 
        for y in range(filterSize): #Apply over all filter
            value += padImg[x+row][y+col]*filterArr[x][y] #Add image multiplied by filter
    return value
#Softmax function
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0) #Calculate softmax
print(apply(filters[1][1],arr1r,0,0))

-0.00142250834637


In [8]:
### Apply filters ###
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])): #Apply over all image
            new_img[y][x] += activation(apply(filterArr,img,y,x)) #Add up all cross-correlations (with ReLU)
    return new_img

In [20]:
### Pooling ###
def pool(img, size):
    pad_w = size-len(img)%size
    pad_h = size-len(img[0])%size
    if pad_w == size:
        pad_w = 0
    if pad_h == size:
        pad_h = 0
    new_img = np.lib.pad(array(img), ((0,pad_w),(0,pad_h)), 'edge') #Pad image
    k = len(new_img)/size #Width of new image 
    l = len(new_img[0])/size #Height of new image
    return new_img.reshape(int(k),size,int(l),size).max(axis=(-1,-3)) #Arrange image into groups of size*size, then get max on axis

In [42]:
### Forward-propagation ###
def forward(img):
    outputLayers = [[img[:,:,0],img[:,:,1],img[:,:,2]]] #Get all the image 
    bias = 0.01
    for num in range(layerNum):
        outputLayers.append([]) #Append a new layer
        for lay in range(layerSize):
            outputLayers[num+1].append(np.zeros((len(outputLayers[num][lay]), len(outputLayers[num][lay][0]))))
            # Add components of the layer
            outputLayers[num][lay] = pool(outputLayers[num][lay], 2)
            # Do pooling
        #lay1 is the layer index you are at and lay2 is the layer index you are going to
        for lay1 in range(layerSize):
            for lay2 in range(layerSize):
                newImage = outputLayers[num+1][lay2] #Get refrence to next layer
                newImage += bias
                #print("S-O: ("+str(len(outputLayers))+", "+str(len(outputLayers[0]))+")")
                #print("S-F: ("+str(len(filters))+", "+str(len(filters[0]))+")")
                applyFilters(outputLayers[num][lay1], newImage, filters[num][layerSize*lay1+lay2])
                #Apply corresponding filter
                if (filterNum*num + layerSize*lay1+lay2) % 20 == 0:
                    print(filterNum*num + layerSize*lay1+lay2)
#                 print(newImage)
    outputLayers.append(softmax(array(outputLayers[len(outputLayers)-1]).flatten())) #Add flattened softmaxed array
    return outputLayers
out = forward(arr1)
print(out)

0
20
40
[[array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.23529412,  1.        ,  0.23529412,  1.        ,  0.23529412,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [ 1.        ,  0.23529412,  0.23529412,  1.        ,  0.23529412,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [ 0.23529412,  1.        ,  0.23529412,  1.        ,  0.35686275,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [ 0.92941176,  0.92941176,  0.92941176,  0.92941176,  0.92941176,
         0.92941176,  0.92941176,  0.92941176,  0.92941176,  0.92941176,
         0.92941176,  0.92941176,  0.92941176],
       [ 1.        ,

In [13]:
### Error + Back-propagation ###
error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets)) # negative sum of log(out) times target- cross-entropy
print(error)
def back(layer, grad):
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize))
    for x in range(filterNum):
        r = 0
#         print("Filter S:"+str(len(filters)))
#         print(layerNum)
        for row in filters[layer-1][x]:
            c=0
            for w in row:
                new_grad[x][r][c]=-np.multiply(error,np.sum(np.multiply(np.multiply(
                    (array(out[layer-1])),
                    w),
                    grad[x][r][c])))
#                 print(-np.divide(error,np.sum(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]))))
#                 print(new_grad[x][r][c])
                filters[layer-1][x][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

3695.01749526
6
5
4
3
2
1


In [30]:
setFilters()
np.seterr(divide='raise')
np.seterr(invalid='raise')
np.seterr(over='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'raise', 'under': 'ignore'}

In [31]:
for x in range(epochs):
    out = forward(arr1)
    error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets))
    print(error)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))

0
20
40
3695.01749526
6
5
4
3
2
1
0
20
40
3695.01749526
6
5
4
3
2
1
0
20
40
3695.01749526
6
5
4
3
2
1
0
20
40
3695.01749526
6
5
4
3
2
1
0
20
40
3695.01749526
6
5
4
3
2
1
0
20
40
3695.01749526
6
5
4
3
2
1
0
20
40
3695.01749526
6
5
4
3
2
1
0
20
40
3695.01749526
6
5
4
3
2
1
0
20
40
3695.01749526
6
5
4
3
2
1
0
20
40
3695.01749526
6
5
4
3
2
1
